# K-Means
K-means algorithm is used to find a pre-determined number of clusters within unlabeled multidimensional dataset. To implement this algorithm, let's import the required packages and generate two-dimensional dataset containing four distinct blobs.

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.datasets.samples_generator import make_blobs
X, y_true = make_blobs(n_samples=300, centers=4,
                       cluster_std=0.90, random_state=0)
plt.scatter(X[:, 0], X[:, 1], s=50);

It is easy to pick four clusters by eye and K-means does this automatically. However, we use 10 random initializations to see which clusters have minimum error.

In [ ]:
kmeans = KMeans(n_clusters=4,n_init=10)
kmeans.fit(X)
y_kmeans = kmeans.predict(X)
plt.figure()
plt.scatter(X[:, 0], X[:, 1], c=y_kmeans, s=50, cmap='viridis')

centers = kmeans.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.5);

### Exercise
As an exercise, you can try making three clusters with 50 random initializations and see the results.

In [ ]:
kmeans = #Enter your code here#
kmeans.fit(X)
y_kmeans = kmeans.predict(X)
plt.figure()
plt.scatter(X[:, 0], X[:, 1], c=y_kmeans, s=50, cmap='viridis')

centers = kmeans.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.5);

## K-means is limited to linear cluster boundaries
K-means can perform poorly if the clusters have complex shapes like the two_moons data.

In [ ]:
from sklearn.datasets import make_moons
X, y = make_moons(200, noise=.05, random_state=0)
labels = KMeans(2, random_state=0).fit_predict(X)
plt.figure()
plt.scatter(X[:, 0], X[:, 1], c=labels,
            s=50, cmap='viridis');

One way to solve this is to use kenrelized k-means that can be implemented by `SpectralClustering` estimator.

In [ ]:
from sklearn.cluster import SpectralClustering
model = SpectralClustering(n_clusters=2, affinity='nearest_neighbors',
                           assign_labels='kmeans')
labels = model.fit_predict(X)
plt.figure()
plt.scatter(X[:, 0], X[:, 1], c=labels,
            s=50, cmap='viridis');

# Principal Component Analysis
One of the most common applications of PCA is visualizing high-dimensional datasets. The Breast Cancer data set is a real-valued multivariate data that consists of two classes, where each class signifies whether a patient has breast cancer or not. The two categories are: malignant and benign. The malignant class has 212 samples, whereas the benign class has 357 samples. It has 30 features shared across all classes as you can see by running the code below.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer
breast = load_breast_cancer()
breast_data = breast.data
breast_labels = breast.target
labels = np.reshape(breast_labels,(569,1))
final_breast_data = np.concatenate([breast_data,labels],axis=1)
breast_dataset = pd.DataFrame(final_breast_data)
features = breast.feature_names
features_labels = np.append(features,'label')
breast_dataset.columns = features_labels
breast_dataset['label'].replace(0, 'Benign',inplace=True)
breast_dataset['label'].replace(1, 'Malignant',inplace=True)
breast_dataset.head()

Now, we can run PCA to project high dimensional data into low dimensional principal components. In this regard, we normalize the data and fit the PCA to find the principal components. By running the following code, you will see how the shape of original dataset is reduced.

In [ ]:
from sklearn.preprocessing import StandardScaler
x = breast_dataset.loc[:, features].values
x_scaled = StandardScaler().fit_transform(x) # normalizing the features
from sklearn.decomposition import PCA
pca_breast = PCA(n_components=2)
principalComponents_breast = pca_breast.fit_transform(x_scaled)
principal_breast_Df = pd.DataFrame(data = principalComponents_breast
             , columns = ['principal component 1', 'principal component 2'])
print("Original shape: {}".format(str(x_scaled.shape)))
print("Reduced shape: {}".format(str(principalComponents_breast.shape)))

By reducing the dimensionality, we should expect loosing some information. Let's see how much of the information is retained and how much is lost.

In [ ]:
print('Explained variation per principal component: {}'.format(pca_breast.explained_variance_ratio_))

This means that the `principal component 1` holds 44.2% of the information and `principal component 2` holds only 19% of the information. Note that by this dimensionality reduction 36.8% of the information is lost. Let's plot the 569 samples along the two principal components.

In [ ]:
plt.figure(figsize=(10,10))
plt.xticks(fontsize=12)
plt.yticks(fontsize=14)
plt.xlabel('Principal Component - 1',fontsize=20)
plt.ylabel('Principal Component - 2',fontsize=20)
plt.title("Principal Component Analysis of Breast Cancer Dataset",fontsize=20)
targets = ['Benign', 'Malignant']
colors = ['r', 'g']
for target, color in zip(targets,colors):
    indicesToKeep = breast_dataset['label'] == target
    plt.scatter(principal_breast_Df.loc[indicesToKeep, 'principal component 1']
               , principal_breast_Df.loc[indicesToKeep, 'principal component 2'], c = color, s = 50)

plt.legend(targets,prop={'size': 15})